In [9]:
import pandas as pd
import glob as glob
import requests
import urllib
from bs4 import BeautifulSoup

In [10]:
def url_to_links_array(url):
    # 
    r = requests.get(url)
    # Beautify
    soup = BeautifulSoup(r.content)
    # find all links on web-page 
    links = soup.findAll('a')
    links_array = []
    for a in links:
        try:
            links_array.append(a['href'])
        except:
            pass
    return links_array

## All the URL links for all the cities

url = "http://insideairbnb.com/get-the-data.html"
airbnb_full_links = url_to_links_array(url)

In [12]:
links_df = pd.DataFrame(airbnb_full_links)
links_df.columns = ['link']
# Get only Data Download Links
links_df = links_df[links_df['link'].str.contains("http://data.insideairbnb.com/united-states/il/chicago/")].reset_index(drop=True)

In [13]:
links_df['country'] = links_df['link'].str.split('/').apply(lambda x: x[3])
links_df['region'] = links_df['link'].str.split('/').apply(lambda x: x[4])
links_df['city'] = links_df['link'].str.split('/').apply(lambda x: x[5])
links_df['date'] = links_df['link'].str.split('/').apply(lambda x: x[6])
links_df['file_name'] = links_df['link'].str.split('/').apply(lambda x: x[8])

In [14]:
links_df.sample(5)

,link,country,region,city,date,file_name
115,http://data.insideairbnb.com/united-states/il/...,united-states,il,chicago,2018-07-18,listings.csv
129,http://data.insideairbnb.com/united-states/il/...,united-states,il,chicago,2018-04-15,listings.csv
136,http://data.insideairbnb.com/united-states/il/...,united-states,il,chicago,2017-05-10,listings.csv
104,http://data.insideairbnb.com/united-states/il/...,united-states,il,chicago,2018-09-14,neighbourhoods.geojson
31,http://data.insideairbnb.com/united-states/il/...,united-states,il,chicago,2019-07-15,listings.csv


In [15]:
import urllib
import gzip
import io
import os

def download_files(dataset):
    for index_val in dataset.index:
        # Get Row
        row = dataset.loc[index_val]
        
        #Get file variables
        download_link = row.link
        country = row.country
        city = row.city
        file_type = row.file_name
        date=row.date
                
        # Directory
        directory = '../data/' + country + '/' + city + '/'
        
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        try:
            #Download details
            file_name = file_type.split('.')[0]+'_'+str(date)

            """Process differently depending on file type"""
            if file_type[-3:] == ".gz":                    
                r = requests.get(download_link, timeout=30, stream=True)
                csv_gz_file = r.content # Content in bytes from requests.get

                f = io.BytesIO(csv_gz_file)
                with gzip.GzipFile(fileobj=f) as fh:
                    # Read unzipped csv to DataFrame
                    dataframe = pd.read_csv(fh, low_memory=False)
                    dataframe.to_csv(directory + file_name + "_full.csv")

            elif file_type[-4:] == ".csv":
                if len(glob.glob(directory + file_name + "_sample.csv")) == 0:
                    open_file = urllib.URLopener()
                    open_file.retrieve(download_link, directory + file_name + "_sample.csv")
            else:
                if len(glob.glob(directory + file_type)) == 0:
                    open_file = urllib.URLopener()
                    open_file.retrieve(download_link, directory + file_type)
        except:
            pass

In [21]:
download_files(links_df)